# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

import numpy as np
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
import tensorflow as tf
print(tf.__version__)


C:\Users\giris\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.16.2


In [3]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
correctly_spelled_words=[]

In [5]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        self.letter_set = sorted(set("".join(self.full_dictionary)))
        self.probabilities = [0] * len(self.letter_set)
        
        self.trigram_probs = self.calculate_trigram_probabilities('word')
    
        self.unigram, self.bigram, self.trigram, self.fourgram, self.fivegram = self.build_n_grams(self.full_dictionary)
    
        self.spelling_model = self.build_spelling_model()
        self.tokenizer, self.model, self.encoder_model, self.decoder_model, self.max_encoder_seq_length = self.build_spelling_model()

        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link
    # ... (previous code remains unchanged)
    
    
    def build_spelling_model(self):
        # Load your dictionary
        correctly_spelled_words = []
        with open('words_250000_train.txt', 'r') as file:
            correctly_spelled_words = [line.strip() for line in file]

        # Generate synthetic data
        synthetic_data = [(self.introduce_misspellings(word), word) for word in correctly_spelled_words]

        # Prepare tokenizer
        input_texts = [d[0] for d in synthetic_data]
        target_texts = [d[1] for d in synthetic_data]
        tokenizer = Tokenizer(char_level=True)
        tokenizer.fit_on_texts(input_texts + target_texts)

        num_encoder_tokens = len(tokenizer.word_index)
        num_decoder_tokens = len(tokenizer.word_index)
        max_encoder_seq_length = max([len(txt) for txt in input_texts])
        max_decoder_seq_length = max([len(txt) for txt in target_texts])

        latent_dim = 256  # Adjust as needed

        # Define encoder
        encoder_inputs = Input(shape=(None, num_encoder_tokens))
        encoder = LSTM(latent_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]
        
        
         # Define encoder model
        encoder_model = Model(encoder_inputs, encoder_states)

        # Define decoder
        decoder_inputs = Input(shape=(None, num_decoder_tokens))
        decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
        decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        decoder_outputs = decoder_dense(decoder_outputs)

        # Define the full model
        model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

       

        # Define decoder model
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)

        return tokenizer, model, encoder_model, decoder_model, max_encoder_seq_length

    # ... (rest of your methods)


    def introduce_misspellings(self, word):
        misspelled_word = list(word)
    
    # Randomly choose an operation: substitute, delete, or insert a character
        operations = ['substitute', 'delete', 'insert']
        operation = random.choice(operations)
     
        if operation == 'substitute':
            index = random.randint(0, len(misspelled_word) - 1)
            misspelled_word[index] = chr(random.randint(97, 122))  # Random lowercase letter
        elif operation == 'delete' and len(misspelled_word) > 1:
            index = random.randint(0, len(misspelled_word) - 1)
            del misspelled_word[index]
        elif operation == 'insert':
            index = random.randint(0, len(misspelled_word))
            misspelled_word.insert(index, chr(random.randint(97, 122)))
    
        return ''.join(misspelled_word)
 
       # Generate synthetic data
    synthetic_data = [(introduce_misspellings(word), word) for word in correctly_spelled_words]
        # ... (your existing introduce_misspellings function) ...

        
        
        
    def decode_sequence(self, input_seq):
        # Encode the input as state vectors.
        states_value = self.encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
        target_seq = np.zeros((1, 1, len(self.tokenizer.word_index)))

    # Sampling loop for a batch of sequences
        decoded_sentence = ''
        stop_condition = False
        while not stop_condition:
            output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = tokenizer.index_word[sampled_token_index]
            decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character.
            if len(decoded_sentence) > max_decoder_seq_length:
                stop_condition = True

        # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, num_tokens))
            target_seq[0, 0, sampled_token_index] = 1.

        # Update states
            states_value = [h, c]

        return decoded_sentence
    def guess_using_spelling_model(self, clean_word):
    # Prepare the input sequence
        input_seq = self.tokenizer.texts_to_sequences([clean_word.replace('_', 'x')])  # Replace underscore with 'x' or any other character
        input_seq = pad_sequences(input_seq, maxlen=self.max_encoder_seq_length, padding='post')
    
    # One-hot encode the input sequence
        input_seq_one_hot = np.zeros((1, input_seq.shape[1], len(self.tokenizer.word_index) + 1))
        for i, seq in enumerate(input_seq):
            input_seq_one_hot[i, np.arange(len(seq)), seq] = 1.

    # Get the corrected word
        corrected_word = self.decode_sequence(input_seq_one_hot)

    # Find the first letter in the corrected word that isn't in the current word and hasn't been guessed
        for letter in corrected_word:
            if letter not in clean_word and letter not in self.guessed_letters:
                return letter

    # If no suitable letter found, fall back to frequency-based guess
        return self.guess_by_overall_frequency()
  
    def decode_sequence(self, input_seq):
      # Encode the input as state vectors.
        states_value = self.encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
        target_seq = np.zeros((1, 1, len(self.tokenizer.word_index) + 1))
    
    # Sampling loop for a batch of sequences
        decoded_sentence = ''
        stop_condition = False
        while not stop_condition:
            output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

        # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = self.tokenizer.index_word[sampled_token_index]
            decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character.
            if len(decoded_sentence) > self.max_encoder_seq_length:
                stop_condition = True

        # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, len(self.tokenizer.word_index) + 1))
            target_seq[0, 0, sampled_token_index] = 1.

        # Update states
            states_value = [h, c]

        return decoded_sentence

#     # Prepare inference model
#     encoder_model = Model(encoder_inputs, encoder_states)

#     decoder_state_input_h = Input(shape=(latent_dim,))
#     decoder_state_input_c = Input(shape=(latent_dim,))
#     decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#     decoder_outputs, state_h, state_c = decoder_lstm(
#         decoder_embedding, initial_state=decoder_states_inputs)
#     decoder_states = [state_h, state_c]
#     decoder_outputs = decoder_dense(decoder_outputs)

#     decoder_model = Model(
#         [decoder_inputs] + decoder_states_inputs,
#         [decoder_outputs] + decoder_states)
 
#     # Test correction
#     input_seq = tokenizer.texts_to_sequences(["I hav a qustion"])
#     input_seq = pad_sequences(input_seq, maxlen=max_encoder_seq_length, padding='post')
#     corrected_text = decode_sequence(input_seq)
#     print("Corrected text:", corrected_text)
#         # ... (your existing decode_sequence function) ...

#     def guess_using_spelling_model(self, clean_word):
#         # Prepare the input sequence
#         input_seq = self.tokenizer.texts_to_sequences([clean_word.replace('_', 'x')])  # Replace underscore with 'x' or any other character
#         input_seq = pad_sequences(input_seq, maxlen=self.max_encoder_seq_length, padding='post')

#         # Get the corrected word
#         corrected_word = self.decode_sequence(input_seq)

#         # Find the first letter in the corrected word that isn't in the current word and hasn't been guessed
#         for letter in corrected_word:
#             if letter not in clean_word and letter not in self.guessed_letters:
#                 return letter

#         # If no suitable letter found, fall back to frequency-based guess
#         return self.guess_by_overall_frequency()







    def short_word_strategy(self, clean_word):
        print("short_word_strategy Executed")
        word_length = len(clean_word)
        unknown_count = clean_word.count('_')
        if word_length <= 7:
            known_count = sum(1 for letter in clean_word if letter != '_')
            
            if known_count == 0:
                return self.find_most_common_letter(self.current_dictionary)
            elif 2<unknown_count<len(clean_word)-2:
                return self.final_strategy(clean_word)
            else:
                # Use a combination of n-gram and spelling model
                ngram_guess = self.calculate_trigram_probabilities(clean_word)
                spelling_guess = self.guess_using_spelling_model(clean_word)
                
                if random.random() < 0.5:  # 50% chance to use each method
                    return spelling_guess
                else:
                    return spelling_guess
        
        return self.guess(clean_word)
    
    
    
    def guess_by_overall_frequency(self):
    # Use the pre-calculated letter frequency from the full dictionary
        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                return letter
        return ''  # Return empty string if all letters have been guessed
    
    def build_n_grams(self, dictionary):
        '''
        build nested dictionary containing occurences for n (1-5) sequences of letters
        unigrams and bigrams have an extra level for length of the word
        for unigram, take only unique letters within each word  
        '''
        unigram = collections.defaultdict(lambda: collections.defaultdict(int))
        bi_gram = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))
        tri_gram = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))
        four_gram = collections.defaultdict(lambda:collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int))))
        five_gram = collections.defaultdict(lambda: collections.defaultdict(lambda:collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))))
        
        # go through each word in the dictionary
        for word in dictionary:
            # check each letter in the dictionary and update the n-gram
            for i in range(len(word) - 4):
                bi_gram[len(word)][word[i]][word[i+1]] += 1
                tri_gram[word[i]][word[i+1]][word[i+2]] += 1
                four_gram[word[i]][word[i+1]][word[i+2]][word[i+3]] += 1
                five_gram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]] += 1
            i = len(word) - 4
            
            # fill out the rest of the n-grams for words too short
            if len(word) == 2:
                bi_gram[len(word)][word[0]][word[1]] += 1
            elif len(word) == 3:
                bi_gram[len(word)][word[0]][word[1]] += 1
                bi_gram[len(word)][word[1]][word[2]] += 1
                tri_gram[word[0]][word[1]][word[2]] += 1
                
            # fill out rest of the (1-4)-grams
            elif len(word) >= 4:
                bi_gram[len(word)][word[i]][word[i+1]] += 1
                bi_gram[len(word)][word[i+1]][word[i+2]] += 1
                bi_gram[len(word)][word[i+2]][word[i+3]] += 1
                tri_gram[word[i]][word[i+1]][word[i+2]] += 1
                tri_gram[word[i+1]][word[i+2]][word[i+3]] += 1
                four_gram[word[i]][word[i+1]][word[i+2]][word[i+3]] += 1
            
            # fill out unigrams
            for letter in set(word):
                unigram[len(word)][letter] += 1
                    
        return unigram, bi_gram, tri_gram, four_gram, five_gram
                    
    
    
    
    
    def recalibrate_n_grams(self):
        '''
        re-tabulates the n-grams after eliminating any incorrectly guessed letters
        updates the dictionary to remove words containing incorrectly guessed letters
        '''
        # updates the dictionary to remove words containing incorrectly guessed letters
        new_dictionary = [word for word in self.full_dictionary if not set(word).intersection(set(self.incorrect_guesses))]
        self.unigram, self.bigram, self.trigram, self.fourgram, self.fivegram = self.build_n_grams(new_dictionary)

    
    def fivegram_probs(self, word):
        ''' 
        Input: the word in the "clean" format with no spaces and a '_' if letter has not been guessed
        Flow: uses tri-gram to calculate the probability of a certain letter appearing in a five-letter sequence for a word of given length
        Output: probabilities for each letter to be used in next level
        '''
                
        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)
        
        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find patterns that have three consecutive letters where one of them is blank
        for i in range(len(word) - 4):
                        
            # case 1: "letter letter letter letter blank"
            if word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] == '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+1]
                anchor_letter_3 = word[i+2]
                anchor_letter_4 = word[i+3]
                
                # calculate occurences of "anchor_letter_1 anchor_letter_2 blank" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4][letter]
                        letter_count[j] += self.fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4][letter]
        
            # case 2: "letter letter letter blank letter"
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] == '_' and word[i+4] != '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+1]
                anchor_letter_3 = word[i+2]
                anchor_letter_4 = word[i+4]
                
                # calculate occurences of "anchor_letter_1 blank anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter][anchor_letter_4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter][anchor_letter_4]
                        letter_count[j] += self.fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter][anchor_letter_4]
               
            # case 3: letter letter blank letter letter
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] == '_' and word[i+3] != '_' and word[i+4] != '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+1]
                anchor_letter_3 = word[i+3]
                anchor_letter_4 = word[i+4]
                
                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3][anchor_letter_4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3][anchor_letter_4]
                        letter_count[j] += self.fivegram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3][anchor_letter_4]
               
            # case 4: letter blank letter letter letter
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] != '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+2]
                anchor_letter_3 = word[i+3]
                anchor_letter_4 = word[i+4]
                
                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3][anchor_letter_4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3][anchor_letter_4]
                        letter_count[j] += self.fivegram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3][anchor_letter_4]
        
            # case 5: blank letter letter letter letter
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] != '_':
                anchor_letter_1 = word[i+1]
                anchor_letter_2 = word[i+2]
                anchor_letter_3 = word[i+3]
                anchor_letter_4 = word[i+4]
                
                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4]
                        letter_count[j] += self.fivegram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4]
        
        # calculate the probabilities of each letter appearing
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count
        
        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.40)
        
        # run the next level down
        return self.fourgram_probs(word)
    
    def fourgram_probs(self, word):
        ''' 
        Input: the word in the "clean" format with no spaces and a '_' if letter has not been guessed
        Flow: uses tri-gram to calculate the probability of a certain letter appearing in a four-letter sequence for a word of given length
        Output: probabilities for each letter to be used in next level
        '''
                
        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)
        
        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find patterns that have three consecutive letters where one of them is blank
        for i in range(len(word) - 3):
                        
            # case 1: "letter letter letter blank"
            if word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] == '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+1]
                anchor_letter_3 = word[i+2]
                
                # calculate occurences of "anchor_letter_1 anchor_letter_2 blank" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter]
                        letter_count[j] += self.fourgram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter]
        
            # case 2: "letter letter blank letter"
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] == '_' and word[i+3] != '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+1]
                anchor_letter_3 = word[i+3]
                
                # calculate occurences of "anchor_letter_1 blank anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3]
                        letter_count[j] += self.fourgram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3]
               
            # case 3: letter blank letter letter
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_' and word[i+3] != '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+2]
                anchor_letter_3 = word[i+3]
                
                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3]
                        letter_count[j] += self.fourgram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3]
               
            # case 4: blank letter letter letter
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_':
                anchor_letter_1 = word[i+1]
                anchor_letter_2 = word[i+2]
                anchor_letter_3 = word[i+3]
                
                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3]
                        letter_count[j] += self.fourgram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3]
        
        # calculate the probabilities of each letter appearing
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count
        
        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.25)
        
        # run the next level down
        return self.calculate_trigram_probabilities(word)

    def calculate_trigram_probabilities(self, word):
        ''' 
        Input: the word in the "clean" format with no spaces and a '_' if letter has not been guessed
        Flow: uses tri-gram to calculate the probability of a certain letter appearing in a three-letter sequence for a word of given length
        Output: probabilities for each letter to be used in next level
        '''
                
        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)
        
        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find patterns that have three consecutive letters where one of them is blank
        for i in range(len(word) - 2):
                        
            # case 1: "letter letter blank"
            if word[i] != '_' and word[i+1] != '_' and word[i+2] == '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+1]
                
                # calculate occurences of "anchor_letter_1 anchor_letter_2 blank" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[anchor_letter_1][anchor_letter_2][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[anchor_letter_1][anchor_letter_2][letter]
                        letter_count[j] += self.trigram[anchor_letter_1][anchor_letter_2][letter]
        
            # case 2: "letter blank letter"
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_':
                anchor_letter_1 = word[i]
                anchor_letter_2 = word[i+2]
                
                # calculate occurences of "anchor_letter_1 blank anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[anchor_letter_1][letter][anchor_letter_2] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[anchor_letter_1][letter][anchor_letter_2]
                        letter_count[j] += self.trigram[anchor_letter_1][letter][anchor_letter_2]
               
            # case 3: blank letter letter
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_':
                anchor_letter_1 = word[i+1]
                anchor_letter_2 = word[i+2]
                
                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[letter][anchor_letter_1][anchor_letter_2] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[letter][anchor_letter_1][anchor_letter_2]
                        letter_count[j] += self.trigram[letter][anchor_letter_1][anchor_letter_2]
        
        # calculate the probabilities of each letter appearing
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count
        
        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.20)
        
        # run the next level down
        return self.bigram_probs(word)
    
    
    def bigram_probs(self, word):
        ''' 
        Input: the word in the "clean" format with no spaces and a '_' if letter has not been guessed
        Flow: uses bi-gram to calculate the probability of a certain letter appearing in a two-letter sequence for a word of given length
              updates the probabilities set in trigram_probs
        Output: probabilities for each letter to be used in next level
        '''
        
        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)
        
        total_count = 0
        letter_count = [0] * len(self.letter_set)
        
        # traverse the word and find either patterns of "letter blank" or "blank letter"
        for i in range(len(word) - 1):
            # case 1: "letter blank"
            if word[i] != '_' and word[i+1] == '_':
                anchor_letter = word[i]
                
                # calculate occurences of "anchor_letter blank" and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.bigram[len(word)][anchor_letter][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.bigram[len(word)][anchor_letter][letter]
                        letter_count[j] += self.bigram[len(word)][anchor_letter][letter]
                            
            # case 2: "blank letter"
            elif word[i] == '_' and word[i+1]!= '_':
                anchor_letter = word[i+1]
                
                # calculate occurences of "blank anchor_letter" and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.bigram[len(word)][letter][anchor_letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.bigram[len(word)][letter][anchor_letter]
                        letter_count[j] += self.bigram[len(word)][letter][anchor_letter]
                                                                    
        # calculate the probabilities of each letter appearing
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.10)
        
        # return letter associated with highest probability
        return self.unigram_probs(word)
    
    
    def unigram_probs(self, word):
        ''' 
        Input: the word in the "clean" format with no spaces and a '_' if letter has not been guessed
        Flow: uses unigram to calculate the probability of a certain letter appearing in a any blank space
              updates the probabilities set in bigram_probs
        Output: letter with the overall highest probability
        '''
                
        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)
        
        total_count = 0
        letter_count = [0] * len(self.letter_set)
        
        # traverse the word and find blank spaces
        for i in range(len(word)):
            # case 1: "letter blank"
            if word[i] == '_':
                                
                # calculate occurences of pattern and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.unigram[len(word)][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.unigram[len(word)][letter]
                        letter_count[j] += self.unigram[len(word)][letter]
                       
        # calculate the probabilities of each letter appearing
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count
                
        # interpolate probabilities
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.05)
        
        # adjust probabilities so they sum to one (not necessary but looks better)
        final_probs = [0] * len(self.letter_set)
        if sum(self.probabilities) > 0:
            for i in range(len(self.probabilities)):
                final_probs[i] = self.probabilities[i] / sum(self.probabilities)
            
        self.probabilities = final_probs
        
        # find letter with largest probability
        max_prob = 0
        guess_letter = ''
        for i, letter in enumerate(self.letter_set):
            if self.probabilities[i] > max_prob:
                max_prob = self.probabilities[i]
                guess_letter = letter
        
        # if no letter chosen from above, pick a random one (extra weight on vowels)
        if guess_letter == '':
            letters = self.letter_set.copy()
            random.shuffle(letters)
            letters_shuffled = ['e','a','i','o','u'] + letters
            for letter in letters_shuffled:
                if letter not in self.guessed_letters:
                    return letter
            
        return guess_letter
    
    
    
    
    def calculate_trigram_probabilities(self,word):
        
        trigram_counts = collections.defaultdict(int)
        bigram_counts = collections.defaultdict(int)
    
        for word in self.full_dictionary:
            padded_word = '_' + word + '_'  # Add padding to consider start and end of words
            for i in range(len(padded_word) - 2):
                trigram = padded_word[i:i+3]
                trigram_counts[trigram] += 1
                bigram_counts[trigram[:2]] += 1
    
        trigram_probs = {}
        for trigram, count in trigram_counts.items():
            bigram = trigram[:2]
            trigram_probs[trigram] = count / bigram_counts[bigram]
    
        return trigram_probs
    
    
    
    
    
    

    def guess_using_trigrams(self, clean_word):
        unknown_positions = [i for i, letter in enumerate(clean_word) if letter == '_']
        padded_word = '_' + clean_word + '_'
    
        letter_scores = collections.defaultdict(float)
    
        for pos in unknown_positions:
            prev_bigram = padded_word[pos:pos+2]
            next_bigram = padded_word[pos+1:pos+3]
        
            for letter in 'abcdefghijklmnopqrstuvwxyz':
                if letter not in self.guessed_letters:
                    prev_trigram = prev_bigram[0] + letter + prev_bigram[1]
                    next_trigram = letter + next_bigram
                
                    prev_score = self.trigram_probs.get(prev_trigram, 0)
                    next_score = self.trigram_probs.get(next_trigram, 0)
                
                    letter_scores[letter] += prev_score + next_score
    
        if letter_scores:
            return max(letter_scores, key=letter_scores.get)
        
        else:
            return ''
        
        
        
        
    

    def find_most_common_letter(self, words):
        letter_count = {}
        for letter in 'abcdefghijklmnopqrstuvwxyz':
            letter_count[letter] = sum(1 for word in words if letter in word)
        return max((letter for letter in letter_count if letter not in self.guessed_letters), 
                   key=letter_count.get, default='')
    
    
    
    

    def intermediate_strategy(self, clean_word):
        # This function implements the strategy for when we have 100 or fewer words
        letter_freq = {}
        for word in self.current_dictionary:
            for letter in set(word):
                if letter not in self.guessed_letters:
                    letter_freq[letter] = letter_freq.get(letter, 0) + 1
        
        # Return the letter that appears in the most words
        return max(letter_freq, key=letter_freq.get) if letter_freq else ''
    
    
    
    

    def calculate_transition_probabilities(self, position='pre'):
        transition_probs = collections.defaultdict(lambda: collections.defaultdict(int))
        total_counts = collections.defaultdict(int)
        
        for word in self.full_dictionary:
            for i in range(len(word) - 1):
                if position == 'pre':
                    transition_probs[word[i]][word[i+1]] += 1
                    total_counts[word[i]] += 1
                else:  # post
                    transition_probs[word[i+1]][word[i]] += 1
                    total_counts[word[i+1]] += 1
        
        for char in transition_probs:
            for next_char in transition_probs[char]:
                transition_probs[char][next_char] /= total_counts[char]
        
        return transition_probs

    def calculate_pair_probabilities(self):
        pair_probs = collections.defaultdict(int)
        total_pairs = 0
        
        for word in self.full_dictionary:
            for i in range(len(word) - 1):
                pair_probs[word[i:i+2]] += 1
                total_pairs += 1
        
        for pair in pair_probs:
            pair_probs[pair] /= total_pairs
        
        return pair_probs
    
    
    
    
    def final_strategy(self, clean_word):
        unknown_positions = [i for i, letter in enumerate(clean_word) if letter == '_']
    
        if len(unknown_positions) == 1:
        # Use the n-gram approach for the last letter
            return self.guess_last_letter_ngram(clean_word)
        elif 6 > len(unknown_positions) > 1:
        # Keep your existing strategy for multiple unknown letters
            return self.guess_last_letter_ngram(clean_word)
        else:
        # Fallback to overall letter frequency if no unknowns (shouldn't happen, but just in case)
            return self.guess_by_overall_frequency()

    def guess_last_letter_ngram(self, clean_word):
        print("Final Strategyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy")
        unknown_position = clean_word.index('_')
        word_length = len(clean_word)
    
    # Reset probabilities
        self.probabilities = [0] * len(self.letter_set)
    
    # Apply 5-gram
        self.fivegram_probs(clean_word)
      
    # Apply 4-gram
        self.fourgram_probs(clean_word)
    
    # Apply 3-gram
        self.calculate_trigram_probabilities(clean_word)
    
    # Apply 2-gram
        self.bigram_probs(clean_word)
    
    # Apply 1-gram
        self.unigram_probs(clean_word)
    
    # Find letter with largest probability
        max_prob = 0
        guess_letter = ''
        for i, letter in enumerate(self.letter_set):
            if self.probabilities[i] > max_prob and letter not in self.guessed_letters:
                max_prob = self.probabilities[i]
                guess_letter = letter
    
    # If no letter chosen from above, pick a random unguessed letter
        if guess_letter == '':
            unguessed = [letter for letter in self.letter_set if letter not in self.guessed_letters]
            if unguessed:
                return final_strategy()
    
        return guess_letter
    
    def final_strategy2(self, clean_word):
        print("stat222222222222222222222222222222222222222")
        unknown_positions = [i for i, letter in enumerate(clean_word) if letter == '_']
        
        if len(unknown_positions) == 2:
            if unknown_positions[1] - unknown_positions[0] > 1:  # Non-adjacent unknowns
                pre_letter = clean_word[unknown_positions[0] - 1] if unknown_positions[0] > 0 else None
                post_letter = clean_word[unknown_positions[1] + 1] if unknown_positions[1] < len(clean_word) - 1 else None
                
                pre_probs = self.calculate_transition_probabilities('pre')
                post_probs = self.calculate_transition_probabilities('post')
                
                combined_probs = {}
                for letter in 'abcdefghijklmnopqrstuvwxyz':
                    if letter not in self.guessed_letters:
                        pre_prob = pre_probs[pre_letter][letter] if pre_letter else 1
                        post_prob = post_probs[post_letter][letter] if post_letter else 1
                        combined_probs[letter] = pre_prob * post_prob
                
                guess_letters = sorted(combined_probs, key=combined_probs.get, reverse=True)[:2]
            else:  # Adjacent unknowns (including end cases)
                if unknown_positions[0] == 0:  # Start of word
                    known_letter = clean_word[unknown_positions[1] + 1]
                    probs = self.calculate_transition_probabilities('pre')
                elif unknown_positions[1] == len(clean_word) - 1:  # End of word
                    known_letter = clean_word[unknown_positions[0] - 1]
                    probs = self.calculate_transition_probabilities('post')
                else:  # Middle of word
                    pre_letter = clean_word[unknown_positions[0] - 1]
                    post_letter = clean_word[unknown_positions[1] + 1]
                    pre_probs = self.calculate_transition_probabilities('pre')
                    post_probs = self.calculate_transition_probabilities('post')
                    pair_probs = self.calculate_pair_probabilities()
                
                combined_probs = {}
                if unknown_positions[0] == 0 or unknown_positions[1] == len(clean_word) - 1:
                    for letter1 in 'abcdefghijklmnopqrstuvwxyz':
                        if letter1 not in self.guessed_letters:
                            for letter2 in 'abcdefghijklmnopqrstuvwxyz':
                                if letter2 not in self.guessed_letters and letter2 != letter1:
                                    if unknown_positions[0] == 0:
                                        prob = probs[known_letter][letter2]
                                    else:
                                        prob = probs[known_letter][letter1]
                                    combined_probs[(letter1, letter2)] = prob
                else:
                    for letter1 in 'abcdefghijklmnopqrstuvwxyz':
                        if letter1 not in self.guessed_letters:
                            for letter2 in 'abcdefghijklmnopqrstuvwxyz':
                                if letter2 not in self.guessed_letters and letter2 != letter1:
                                    pre_prob = pre_probs[pre_letter][letter1]
                                    post_prob = post_probs[post_letter][letter2]
                                    pair_prob = pair_probs.get(letter1 + letter2, 1e-10)  # Small value to avoid division by zero
                                    combined_probs[(letter1, letter2)] = (pre_prob * post_prob) / pair_prob
                
                guess_letters = sorted(combined_probs, key=combined_probs.get, reverse=True)[0]
            
            self.final_strategy_second_guess = guess_letters[1]
            return guess_letters[0]
        elif hasattr(self, 'final_strategy_second_guess'):
            guess_letter = self.final_strategy_second_guess
            del self.final_strategy_second_guess
            return guess_letter
        else:
            # Fallback to previous strategy if we have only one unknown
            letter_freq = {}
            for word in self.current_dictionary:
                for pos in unknown_positions:
                    letter = word[pos]
                    if letter not in self.guessed_letters:
                        letter_freq[letter] = letter_freq.get(letter, 0) + 1
            return max(letter_freq, key=letter_freq.get) if letter_freq else ''
        
        
        


    def guess(self, word):
        clean_word = word[::2]
        word_length = len(clean_word)

        # If this is the first guess, create a new dictionary with words of the same length
        if not self.guessed_letters:
            self.current_dictionary = [w for w in self.full_dictionary if len(w) == word_length]
        else:
        # Update the dictionary based on the last guess
            last_guess = self.guessed_letters[-1]
            new_dictionary = []
            for w in self.current_dictionary:
                if last_guess in clean_word:
                    if all((w[i] == last_guess) == (clean_word[i] == last_guess) for i in range(word_length)):
                        new_dictionary.append(w)
                else:
                    if last_guess not in w:
                        new_dictionary.append(w)
            self.current_dictionary = new_dictionary
        # Count unknown positions
        unknown_count = clean_word.count('_')
        # Use the main strategy until we're left with 100 or fewer words
        if len(clean_word)>10:
        
            if unknown_count > len(clean_word)-3:
                guess_letter = self.find_most_common_letter(self.current_dictionary)
                    # Switch to the intermediate strategy when we have 100 or fewer words
#         elif unknown_count > 1:
#             guess_letter = self.final_strategy2(clean_word)

        # Switch to the final strategy when we have 2 or fewer unknown positions
            else:
                guess_letter = self.final_strategy(clean_word)
        
        elif word_length <= 7:
            if unknown_count>len(clean_word)-2:
                guess_letter = self.find_most_common_letter(self.current_dictionary)
            elif 2<unknown_count<len(clean_word)-2:
                guess_letter = self.final_strategy(clean_word)
            else:    
                return self.short_word_strategy(clean_word)

        
        
        else:
            if unknown_count > len(clean_word)-4:
                guess_letter = self.find_most_common_letter(self.current_dictionary)
                    # Switch to the intermediate strategy when we have 100 or fewer words
#         elif unknown_count > 1:
#             guess_letter = self.final_strategy2(clean_word)

        # Switch to the final strategy when we have 2 or fewer unknown positions
            else:
                guess_letter = self.final_strategy(clean_word)
           
        # If we can't find a new letter to guess, guess a random unguessed letter
        if not guess_letter:
            alphabet = 'abcdefghijklmnopqrstuvwxyz'
            unguessed = [letter for letter in alphabet if letter not in self.guessed_letters]
            guess_letter = random.choice(unguessed) if unguessed else ''

        if guess_letter:
            self.guessed_letters.append(guess_letter)
            
        else:
            print("No more letters to guess.")
            
        
        return guess_letter
     
        
# Replace with your own "guess" function here #
###############################################
##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [6]:
api = HangmanAPI(access_token="7af05f308f446e991c5c4a9b4775d9", timeout=2000)
 

In [8]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: e2ea9f6653e2. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'e2ea9f6653e2', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ e _ '}
Guessing letter: r
Sever response: {'game_id': 'e2ea9f6653e2', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ e _ '}
Guessing letter: s
Sever response: {'game_id': 'e2ea9f6653e2', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ e s '}
short_word_strategy Executed


ValueError: Input 0 of layer "functional_3" is incompatible with the layer: expected shape=(None, None, 26), found shape=(1, 29, 27)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [21]:
# Assuming the provided code is part of a larger script or function

# Set the target practice success rate
target_success_rate = 0.50

# Continue the loop until the practice success rate reaches the target
while practice_success_rate < target_success_rate and total_practice_runs < 610:
    # Start a new game
    api.start_game(practice=1, verbose=True)

    # Get updated game stats
    [total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] = api.my_status()

    # Calculate the practice success rate
    practice_success_rate = total_practice_successes / total_practice_runs

    # Print the current status
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

# Print a message when the loop exits
if practice_success_rate >= target_success_rate:
    print("Achieved the target practice success rate of %.3f!" % target_success_rate)
else:
    print("Exceeded the maximum allowed practice games (100,000) without reaching the target success rate.")

NameError: name 'total_practice_runs' is not defined

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [111]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

ZeroDivisionError: division by zero